# 🔄 Deteksi Code-Switching dengan Naive Bayes & TF-IDF

**Judul Skripsi:** Implementasi Naive Bayes dan TF-IDF untuk Deteksi Kode-Switching Bahasa Indonesia-Inggris pada Teks Media Sosial

---

## 📚 Library yang Digunakan

| Library | Fungsi |
|---------|--------|
| `pandas` | Manipulasi data tabular |
| `numpy` | Operasi numerik |
| `re` | Regular expression untuk preprocessing |
| `matplotlib` | Visualisasi confusion matrix |
| `seaborn` | Heatmap visualization |
| `sklearn` | Machine learning pipeline |

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print("Library berhasil dimuat!")

---

# 📌 TAHAP 1: WEAK SUPERVISION (PELABELAN OTOMATIS)

Pada tahap ini, kita akan melabeli data secara otomatis menggunakan **lexicon-based approach**.

## 1️⃣ Memuat Dataset Mentah

In [ ]:
# Untuk Google Colab: Upload file terlebih dahulu
# from google.colab import files
# uploaded = files.upload()

try:
    df = pd.read_csv('codeswitch_emotion.csv', on_bad_lines='skip')
    print(f"[INFO] Dataset berhasil dimuat: {len(df)} baris")
    display(df.head())
except FileNotFoundError:
    print("[ERROR] File tidak ditemukan. Menggunakan data dummy.")
    df = pd.DataFrame({'tweet': [
        "Aku stuck banget sama deadline tugas", 
        "I love you so much", 
        "Makan nasi goreng enak di kantin"
    ]})

## 2️⃣ Definisi Lexicon (Kamus Kata)

Lexicon berisi kata-kata penanda untuk masing-masing bahasa.

**Catatan Penting:**
- Kata-kata **ambiguous** (yang sering muncul di kedua konteks) telah dihapus
- Contoh kata yang dihapus: `in`, `on`, `as`, `be`, `so`, `good`, `bad`, `not`, `no`

In [ ]:
# Lexicon Bahasa Indonesia
indo_markers = {
    # Kata Ganti & Tunjuk
    'aku', 'kamu', 'dia', 'kita', 'mereka', 'ini', 'itu', 'sini', 'situ', 'sana',
    'gue', 'lu', 'lo', 'gw', 'anda', 'saya', 'kalian',
    # Kata Sambung & Depan
    'dan', 'atau', 'tapi', 'tetapi', 'karena', 'krn', 'jika', 'kalau', 'kalo', 
    'yang', 'yg', 'dari', 'pada', 'dalam', 'untuk', 'utk', 'buat', 
    'dengan', 'dgn', 'sama', 'bisa', 'dapat', 'akan', 'ingin', 'mau', 'sudah', 
    'telah', 'sedang', 'lagi', 'lg', 'masih', 'belum', 'blm',
    # Kata Tanya & Seru
    'apa', 'kenapa', 'knp', 'mengapa', 'gimana', 'bagaimana', 'siapa', 'kapan', 
    'dimana', 'kok', 'sih', 'dong', 'deh', 'kan', 'yuk', 'wkwk', 'hehe', 'haha',
    'wah', 'nah', 'loh', 'lah', 'kah', 'pun',
    # Kata Kerja & Sifat Umum
    'makan', 'minum', 'tidur', 'jalan', 'lihat', 'dengar', 'baca', 'tulis', 
    'beli', 'jual', 'bayar', 'kerja', 'suka', 'cinta', 'benci', 'marah',
    'senang', 'sedih', 'takut', 'berani', 'malu', 'bangga', 'bagus', 'jelek',
    'baik', 'jahat', 'benar', 'salah', 'cepat', 'lambat', 'mahal', 'murah',
    'terima', 'kasih', 'tolong', 'maaf', 'selamat', 'pagi', 'siang', 'malam',
    'rumah', 'orang', 'anak', 'hari', 'tahun', 'waktu', 'uang', 'harga',
    'tidak', 'tak', 'gak', 'ga', 'nggak', 'bukan', 'jangan', 'usah', 'udah'
}

print(f"Total kata Indonesia: {len(indo_markers)}")

In [ ]:
# Lexicon Bahasa Inggris (kata ambiguous dihapus)
eng_markers = {
    # Pronouns & Prepositions
    'i', 'you', 'he', 'she', 'we', 'they', 'this', 'that', 'these', 'those',
    'my', 'your', 'his', 'her', 'our', 'their', 'mine', 'yours',
    'for', 'with', 'from', 'about', 'into', 'through', 'after', 'over', 'between', 'against',
    # Conjunctions & Auxiliary Verbs
    'and', 'because', 'when', 'where', 'why', 'how',
    'is', 'am', 'are', 'was', 'were', 'been', 'being',
    'have', 'has', 'had', 'does', 'did', 'done',
    'can', 'could', 'will', 'would', 'shall', 'should', 'may', 'might', 'must',
    # Common Verbs
    'want', 'need', 'know', 'think', 'take', 'see', 'get', 'give', 'come',
    'make', 'look', 'use', 'find', 'tell', 'ask', 'seem', 'feel', 'try',
    'leave', 'call', 'drink', 'eat', 'sleep', 'run', 'walk', 'talk', 'speak',
    'say', 'help', 'start', 'stop', 'move', 'write', 'read', 'pay', 'buy', 'sell',
    # Adjectives & Adverbs
    'great', 'high', 'low', 'big', 'small', 'long', 'short',
    'new', 'old', 'right', 'wrong', 'happy', 'sad', 'angry', 'afraid', 'brave',
    'beautiful', 'ugly', 'expensive', 'cheap', 'fast', 'slow', 'hard', 'soft',
    'actually', 'literally', 'basically', 'totally', 'honestly', 'probably',
    'maybe', 'please', 'thanks', 'sorry', 'excuse', 'hello', 'bye',
    'yeah', 'yep', 'nope', 'never', 'always', 'ever',
    'people', 'life', 'man', 'woman', 'love', 'really', 'very', 'just'
}

print(f"Total kata Inggris: {len(eng_markers)}")

## 3️⃣ Fungsi Pelabelan Otomatis (Ratio-Based)

**Algoritma Pelabelan:**

1. Hitung jumlah kata Indonesia (`id_score`) dan Inggris (`en_score`)
2. Hitung rasio masing-masing bahasa
3. Tentukan label berdasarkan kriteria:

| Kondisi | Label |
|---------|-------|
| `id_score >= 2` AND `en_score >= 2` AND rasio seimbang (25-75%) | **MIX** |
| `en_ratio > 0.6` OR (`en_score >= 2` AND `id_score == 0`) | **EN** |
| Lainnya | **ID** |

In [ ]:
def automated_labeling(text):
    """
    Pelabelan otomatis dengan threshold berbasis RASIO.
    MIX hanya jika kedua bahasa cukup seimbang (25-75%) DAN minimal 2 kata masing-masing.
    """
    if not isinstance(text, str): 
        return 'ID'
    
    text_clean = text.lower()
    text_clean = re.sub(r'[^a-z\s]', ' ', text_clean)
    words = text_clean.split()
    
    if len(words) == 0: 
        return 'ID'
    
    word_set = set(words)
    id_score = len(word_set.intersection(indo_markers))
    en_score = len(word_set.intersection(eng_markers))
    
    total_markers = id_score + en_score
    
    # Jika tidak ada marker sama sekali, default ke ID
    if total_markers == 0: 
        return 'ID'
    
    # Hitung rasio
    id_ratio = id_score / total_markers
    en_ratio = en_score / total_markers
    
    # MIX: kedua bahasa harus cukup seimbang (25-75%) DAN minimal 2 kata masing-masing
    if id_score >= 2 and en_score >= 2 and 0.25 <= id_ratio <= 0.75:
        return 'MIX'
    # EN: mayoritas marker adalah English
    elif en_ratio > 0.6 or (en_score >= 2 and id_score == 0):
        return 'EN'
    # ID: default atau mayoritas Indonesia
    else:
        return 'ID'

# Test fungsi
test_cases = [
    "Aku capek sekali",
    "I love you so much",
    "Aku stuck banget sama deadline tugas"
]

print("Test Pelabelan:")
for text in test_cases:
    print(f"  '{text}' → {automated_labeling(text)}")

## 4️⃣ Menerapkan Pelabelan ke Dataset

In [ ]:
print("[...] Sedang menjalankan algoritma pelabelan otomatis...")
df['label_bahasa'] = df['tweet'].apply(automated_labeling)
print("[✓] Pelabelan selesai!")

# Tampilkan statistik
print("\n📊 Distribusi Label:")
stats = df['label_bahasa'].value_counts()
total = len(df)
for label, count in stats.items():
    print(f"  {label}: {count} ({count/total:.1%})")

In [ ]:
# Visualisasi distribusi label
plt.figure(figsize=(8, 5))
colors = ['#4CAF50', '#2196F3', '#FF9800']
df['label_bahasa'].value_counts().plot(kind='bar', color=colors, edgecolor='black')
plt.title('Distribusi Label Bahasa', fontsize=14, fontweight='bold')
plt.xlabel('Label')
plt.ylabel('Jumlah')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

## 5️⃣ Menyimpan Dataset Hasil Pelabelan

In [ ]:
output_csv = 'dataset_hasil_pelabelan.csv'
df[['tweet', 'label_bahasa']].to_csv(output_csv, index=False)
print(f"[✓] File '{output_csv}' berhasil disimpan!")

# Untuk Google Colab: Download file
# from google.colab import files
# files.download(output_csv)

---

# 📌 TAHAP 2: PELATIHAN MODEL NAIVE BAYES

Pada tahap ini, kita akan melatih model klasifikasi menggunakan:
- **TF-IDF Vectorizer**: Mengubah teks menjadi vektor numerik
- **Multinomial Naive Bayes**: Algoritma klasifikasi probabilistik

## 1️⃣ Preprocessing Teks

In [ ]:
def clean_text_final(text):
    """
    Membersihkan teks untuk input model.
    - Lowercase
    - Hapus username dan URL placeholder
    - Hapus karakter non-alfanumerik
    """
    text = str(text).lower()
    text = re.sub(r'\[username\]|\[url\]', '', text)
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return text.strip()

df['text_clean'] = df['tweet'].apply(clean_text_final)
print("[✓] Preprocessing selesai!")
display(df[['tweet', 'text_clean', 'label_bahasa']].head())

## 2️⃣ Split Data Training dan Testing

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df['text_clean'], 
    df['label_bahasa'], 
    test_size=0.2, 
    random_state=42,
    stratify=df['label_bahasa']
)

print(f"Data Training: {len(X_train)} sampel")
print(f"Data Testing : {len(X_test)} sampel")
print(f"\nDistribusi Training:")
print(y_train.value_counts())

## 3️⃣ Oversampling untuk Menyeimbangkan Kelas

Kita menggunakan **oversampling** untuk mengatasi ketidakseimbangan kelas.
Setiap kelas akan didup likasi hingga memiliki jumlah yang sama dengan kelas mayoritas.

In [ ]:
print("[...] Melakukan oversampling untuk menyeimbangkan kelas...")

train_df = pd.DataFrame({'text': X_train.values, 'label': y_train.values})

# Hitung jumlah sampel per kelas
class_counts = train_df['label'].value_counts()
max_count = class_counts.max()
print(f"\nSebelum oversampling:")
print(class_counts)

# Oversample setiap kelas agar seimbang
balanced_dfs = []
for label in class_counts.index:
    class_df = train_df[train_df['label'] == label]
    oversampled = class_df.sample(n=max_count, replace=True, random_state=42)
    balanced_dfs.append(oversampled)

train_balanced = pd.concat(balanced_dfs).sample(frac=1, random_state=42).reset_index(drop=True)
X_train = train_balanced['text']
y_train = train_balanced['label']

print(f"\nSesudah oversampling:")
print(y_train.value_counts())
print(f"\n[✓] Data training diseimbangkan: {len(train_balanced)} sampel ({max_count} per kelas)")

## 4️⃣ Melatih Model

**Pipeline:**
1. **TfidfVectorizer**: `ngram_range=(1,2)`, `max_features=5000`
2. **MultinomialNB**: `alpha=0.1` (smoothing parameter)

In [ ]:
model = make_pipeline(
    TfidfVectorizer(ngram_range=(1, 2), max_features=5000), 
    MultinomialNB(alpha=0.1)
)

print("[...] Sedang melatih model...")
model.fit(X_train, y_train)
print("[✓] Model berhasil dilatih!")

---

# 📌 TAHAP 3: EVALUASI & VISUALISASI

## 1️⃣ Menghitung Akurasi

In [ ]:
y_pred = model.predict(X_test)
acc_score = accuracy_score(y_test, y_pred)

print("\n" + "*"*40)
print(" AKURASI MODEL ".center(40, "*"))
print(f"{acc_score:.2%}".center(40))
print("*"*40)

## 2️⃣ Classification Report

| Metrik | Deskripsi |
|--------|----------|
| **Precision** | Dari prediksi positif, berapa yang benar |
| **Recall** | Dari label sebenarnya, berapa yang terdeteksi |
| **F1-Score** | Harmonic mean dari precision dan recall |

In [ ]:
print("\n📊 Detail Laporan Klasifikasi:")
print(classification_report(y_test, y_pred, zero_division=0))

## 3️⃣ Confusion Matrix

In [ ]:
cm = confusion_matrix(y_test, y_pred, labels=['ID', 'EN', 'MIX'])

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['ID', 'EN', 'MIX'], 
            yticklabels=['ID', 'EN', 'MIX'])
plt.title('Confusion Matrix: Deteksi Code-Switching', fontsize=14, fontweight='bold')
plt.ylabel('Label Asli (Weak Supervision)')
plt.xlabel('Prediksi Model')
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=150)
print("[✓] Gambar 'confusion_matrix.png' berhasil disimpan.")
plt.show()

## 4️⃣ Test Prediksi Manual

In [ ]:
test_sentences = [
    "Aku capek sekali",
    "I love you so much",
    "you and they",
    "Aku stuck banget sama deadline tugas",
    "Which is sebenernya dia fine aja"
]

print("\n🔍 Test Prediksi Model:")
print("-" * 60)
for sentence in test_sentences:
    clean = clean_text_final(sentence)
    pred = model.predict([clean])[0]
    prob = model.predict_proba([clean])[0]
    print(f"'{sentence}'")
    print(f"  → Prediksi: {pred}")
    print(f"  → Probabilitas: ID={prob[1]:.1%}, EN={prob[0]:.1%}, MIX={prob[2]:.1%}")
    print()